拟牛顿方法不计算海瑟矩阵$\nabla^2f(x)$, 而是构造其近似矩阵$B^k$或者其逆的近似矩阵$H^k$. 我们希望$B^k$或者$H^k$保留了一些海瑟矩阵的性质, 例如使得$d^k$仍然为下降方向.

### 割线方程
设$f(x)$是二次可微的, 向量值函数$\nabla f(x)$在点$x^{k+1}$处的近似为：
$$\nabla f(x)=\nabla f(x^{k+1})+\nabla^2f(x^{k+1})(x-x^{k+1})+O(||x-x^{k+1}||^2)$$
令 $x = x^k, s^k = x^{k+1} − x^k$ 及 $y^k = \nabla f(x^{k+1})− \nabla f(x^k)$, 得到
$$\nabla^2f(x^{k+1})s^k+O(||s^k||^2)=y^k$$
不考虑高阶项的时候, 我们可以得到海瑟矩阵的近似$B^{k+1}$满足：
$$y^{k}=B^{k+1}s^{k}\tag{1}$$
或者其逆矩阵的近似矩阵$H^{k+1}$:
$$s^{k}=H^{k+1}y^k\tag{2}$$
我们得到的近似矩阵$B^k$仍然需要保证它是正定的！对其两边同时左乘$(s^k)^T$得到$(s^k)^Ty^{k}=(s^k)^TB^{k+1}s^{k}$, 从而有条件:
$$(s^k)^Ty^{k}>0\tag{3}$$
对于一般的目标函数, 采取Wolfe准则就会保证曲率条件(3)成立！理由：
$$\nabla f(x^{k+1})^Ts^k\geq c_2\nabla f(x^k)^Ts^k\\
(y^k)^Ts^k\geq(c_2-1)\nabla f(x^{k})^Ts^k>0$$
这是因为$c_2<1$并且$s^k=\alpha_kd^k$是下降方向.

### 一般框架 
![](1.png)

### BFGS 公式
我们这样计算近似矩阵,设$B^{k+1}=B^{k}+auu^{T}+bvv^T$, 其中$a,b\in R$待定, 根据割线方程(1):
$$B^{k+1}s^{k}=y^k=(B^{k}+auu^{T}+bvv^T)s^k$$
整理一下就有：$$(au^Ts^k)u+(bv^Ts^T)v=y^k-B^ks^k$$
直接让两边系数相等就可以得到$u,v,a,b$的公式了, 然后就可以得到$B^k$到$B^{k+1}$的更新公式
$$B^{k+1}=B^{k}+\frac{y^{k}(y^{k})^T}{(s^k)^Ty^k}-\frac{B^ks^k(B^ks^k)^T}{(s^k)^TB^ks^k}.\tag{4}$$
在问题求解过程中，条件(3) 不一定会得到满足，此时应该使用 Wolfe 准则的线搜索来迫使条件(3)成立. 我们根据这个式子并且假设$H^k=(B^k)^{-1}$可以得到：
$$H^{k+1}=(1-\rho_ky^k(s^k)^T)^TH^k(I-\rho_ky^k(s^k)^T)+\rho_ks^k(s^k)^T\tag{5}$$
其中$\rho_k=\frac{1}{(s^k)^Ty^k}$

### 有限内存的BFGS公式
首先我们引入新的记号去表示$H^{k}$的更新公式：
$$H^{k+1}=(V^k)^TH^kV^k+\rho_ks^k(s^k)^T,\tag{6}$$
其中
$$\rho_k=\frac{1}{(y^k)^Ts^k},\quad V^k=I-\rho_ky^k(s^k)^T$$
更新(6)式可以发现它有递归的特点, 于是我们可以展开$m$次

![](2.png)

很明显$H^{k-m}$还是无法显示求出来, 这个式子也不能无限展开. 我们只用一个近似的$H^{k-m}$去近似$H^{k-m}$, 这样这个式子就方便计算了. 这里给出一个算法.

![](3.png)
![](4.png)

### 应用举例
#### 逻辑回归问题
$$\min_{x\in R^n} l(x)= \frac{1}{m}\sum_{i=1}^{m}ln(1+exp(-b_ia_i^Tx))+\lambda||x||_2^2$$


In [57]:
import numpy as np
import matplotlib.pyplot as plt

In [58]:
m = 5
n = 5
A = np.random.randint(0, 10, (m,n))
B = np.random.randint(0, 10, m)
Lambda = 1/(100*m)

def sum(x):
    s = 0
    for i in range(m):
        s += np.log(1+np.exp(-B[i]*A[i]@x))
    return s
f = lambda x : (1/m)*sum(x) + Lambda*x@x

def grad_sum(x):
    s = 0
    for i in range(m):
        s += (1 - 1/(1+np.exp(-B[i]*A[i]@x))) * B[i] * A[i]
    return s
grad_f = lambda x : -(1/m)*grad_sum(x) + 2*Lambda*x

In [70]:
class quasi_Newton_method():
    def __init__(self, f, grad_f, step):
        self.f = f
        self.grad_f = grad_f
        self.step = step
    ## Wolfe criterion
    def Wolfe(self, x, d):
        alpha = 1
        while self.f(x + alpha * d) > self.f(x) + 0.1 * alpha * self.grad_f(x)@d or self.grad_f(x + alpha * d)@d < 0.9 * self.grad_f(x)@d:
            alpha = 0.5 * alpha
            if alpha < 0.01:
                break
        return alpha
    ## SR1 iteration scheme
    def SR1(self, s, y, H):
        H += ((s - H@y) @ (s - H@y)) / ((s - H@y)@y)
        return H
    ## BFGS method
    def BFGS(self, s, y, H):
        rho = 1/(s@y)
        H = np.transpose(np.eye(m) - rho * np.outer(s,y))@H@(np.eye(m) - rho * np.outer(s,y)) + rho * s@s
        return H
    ## quasi Newton method by SR1
    def solve(self):
        ## init x_0 and H_0 (or B_0)
        H = np.eye(m)
        x = np.linspace(1,10,m)
        # init the result of this method
        result = np.zeros(self.step) 
        for k in range(self.step):
            # calculate the k step's direction
            d = -(H@self.grad_f(x))
            # calculate the k step's size
            alpha =self.Wolfe(x, d)
            # calculate H_k+1
            x_next = x + alpha * d
            s = x_next - x
            y = self.grad_f(x_next) - self.grad_f(x)
            #H = self.SR1(s, y, H)
            H = self.BFGS(s, y, H)
            # store the result
            result[k] = grad_f(x_next)@grad_f(x_next)
            # init the next step
            x = x_next
        return result

In [76]:
sol = quasi_Newton_method(f, grad_f, 100)

In [77]:
r = sol.solve()

C:\User\Shen\AppData\Local\Temp\ipykernel_18896\3625004407.py:10: RuntimeWarning: overflow encountered in exp
  s += np.log(1+np.exp(-B[i]*A[i]@x))


In [78]:
r

array([3.22979813e-03, 1.15024061e-03, 8.57196109e-04, 8.31738266e-04,
       8.85079406e-04, 8.08482852e-04, 7.97760769e-04, 7.92909245e-04,
       7.90958379e-04, 8.74860978e-04, 7.70256408e-04, 7.49779433e-04,
       7.48066207e-04, 9.05167538e-04, 7.35995404e-04, 7.29724717e-04,
       7.28315113e-04, 7.20345568e-04, 7.16654572e-04, 5.54765809e-02,
       3.65272881e-03, 6.66150644e-04, 6.13302073e-04, 5.67786306e-04,
       4.89680043e-04, 4.63877235e-04, 4.56727221e-04, 5.34073807e-04,
       4.14803472e-04, 3.76083854e-04, 3.75421044e-04, 1.26086796e-04,
       4.37002441e-05, 2.82437962e-05, 5.98120922e-04, 1.56370781e-05,
       3.00154279e-05, 9.74096447e-06, 8.52443722e-06, 7.43810850e-06,
       1.53666707e-04, 5.68815434e-06, 2.37601360e-04, 3.42681273e-06,
       8.96910014e-06, 2.55182590e-06, 3.43055819e-06, 1.90744126e-06,
       2.72490229e-06, 1.63612693e-06, 1.96816383e-06, 1.50299486e-06,
       2.10453102e-06, 1.43436380e-06, 1.88177393e-06, 1.17243026e-06,
      